In [9]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [10]:
token = '4e6e771d37dbcbcfcc3b53d291a274d3ae21560a2e81f058a7c177aff044b5141941e89aff1fead50be4f'

In [11]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [12]:
vk.messages.send(
    chat_id=1,
    random_id=2,
    message='Matrix has you ...')

0

In [13]:
df = pd.read_csv('/home/jupyter-an.karpov/shared/ads_data.csv.zip', compression='zip')

PermissionError: [Errno 13] Permission denied: '/home/jupyter-an.karpov/shared/ads_data.csv.zip'

In [ ]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'})

In [ ]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [ ]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click / ad_data.view)

In [ ]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [ ]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [ ]:
path = '/home/jupyter-an.karpov/lesson_7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

In [ ]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [ ]:
saved_file

In [ ]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [ ]:
attachment

In [ ]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR', 
    attachment = attachment
)

In [14]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'anatoly1804@gmail.com'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('heroic-venture-268009-1dfbcc34e5fa.json', scope)
gs = gspread.authorize(credentials)

In [ ]:
# Name of the table in google sheets, 
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'to_sequence'  # Your table
# Get this table
work_sheet = gs.open(table_name)

In [ ]:
# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

In [ ]:
data

In [ ]:
headers = data.pop(0)

In [ ]:
df = pd.DataFrame(data, columns=headers)

In [ ]:
df

In [ ]:
df.sort_values('price', ascending=False)

In [ ]:
sheet1.append_row([500, 'group_4'])

In [ ]:
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)
spreadsheet_name = 'A new spreadsheet'
sheet = 'KarpovCorses2'
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

In [ ]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'

In [ ]:
vistis_url = url + visits

In [ ]:
vistis_request = requests.get(vistis_url)

In [ ]:
vistis_request

In [ ]:
json_data = json.loads(vistis_request.text)

In [ ]:
json_data['data']

In [ ]:
y_df = pd.DataFrame([(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']], columns=['date', 
                                                        'visits'])

In [ ]:
spreadsheet_name = 'A new spreadsheet'
sheet = 'Yandex_visits'
d2g.upload(y_df, spreadsheet_name, sheet, credentials=credentials, row_names=True)